In [ ]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

import requests
import json
import os

def getInfoDicFromJavhoo(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("img", class_="alignnone size-full")['src']
    fullTitle = bsSoup.find("h1", class_="h3-size entry-title").get_text()
    title = fullTitle[fullTitle.find(' ')+1:] 
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", class_="project_info")  
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :')
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                else:
                    body = ""
                valueDic[header] = body
                p = p.find_next_sibling('p')
        else:
            header = p.get_text().strip(' :')
            body = []
            p = p.find_next_sibling('p')
            if (p.contents[0]['class'] == ['genre']):
                 for child in p.contents[0]:
                    body.append(child.get_text())
            p = p.find_next_sibling('p')
            valueDic[header] = body

    valueDic['網站'] = 'Javhoo'
    return valueDic


def getDicFromJavhoo(avID):
    url = "https://www.javhoo.com/av/" + avID.lower()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavhoo(soup)



def getInfoDicFromJavBus(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("a", {'class': "bigImage"}).find('img')['src']
    fullTitle = bsSoup.find("div", {'class': "container"}).find('h3').get_text()
    title = fullTitle[fullTitle.find(' ')+1:]  # title might contains actress name, need to remove it later
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", {'class': "col-md-3 info"})   
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :\n')
#                print('header is: ', header)
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                elif (len(p) == 3):
                    body = p.contents[2].get_text() # try&error, the last element contains data
                else:
                    body = ""
#                print('body is: ', body)
                if header == '識別碼':
                    header = '番號'  # change JavBus naming to Javhoo naming
                valueDic[header] = body.strip(' :\n')
                p = p.find_next_sibling('p')
        elif p.has_attr('class') and p['class'] == ['header']:
            # parsing genre tag for JavBus
            header = p.get_text().strip(' :\n')
#            print('header2 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            for item in p:
                if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                    body.append(item.get_text().strip(' :\n'))           
#            print('body2 is: ', body)
            valueDic[header] = body
            p = p.find_next_sibling('p')            
        elif p.has_attr('class') and p['class'] == ['star-show']:
            # parsing actor for JavBus
            header = p.get_text().strip(' :\n')
#            print('header3 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            if p:
                for item in p:            
                    if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                        body.append(item.get_text().strip(' :\n'))
                p = p.find_next_sibling('p')  
#            print('body3 is: ', body)
            valueDic[header] = body

    valueDic['網站'] = 'JavBus'
    return valueDic


def getDicFromJavBus(avID):
    url = "https://www.javbus.com/" + avID.upper()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavBus(soup)




def getInfoDicFromJavLib(bsSoup):
    valueDic ={}

    #check if multiple items replied
    vtlist = bsSoup.find("div", {'class': "videothumblist"})
    if vtlist:
        multi = vtlist.find_all('div', {'class': 'video'})
        if multi:
            valueDic['多結果'] = 'Multiple returns'
            valueDic['網站'] = 'JavLibrary'            
        else:
        #    not found on JavLibrary
            pass
        return valueDic    
    
    coverPath = bsSoup.find("img", {'id': "video_jacket_img"})
    if not coverPath:
        return valueDic
    else:
        coverPath = coverPath['src']
    fullTitle = bsSoup.find("div", {'id': "video_title"}).find('h3').get_text()
    title = fullTitle[fullTitle.find(' ')+1:]  # title might contains actress name, need to remove it later
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    idd = bsSoup.find("div", {'id': "video_id"}).find("td", {'class', "text"}).get_text()
    valueDic['番號'] = idd.strip(' :\n')
    director = bsSoup.find("div", {'id': "video_director"}).find("td", {'class', "text"}).get_text()
    valueDic['導演'] = director.replace(u'\xa0', ' ').strip(' :\n')
    label = bsSoup.find("div", {'id': "video_label"}).find("td", {'class', "text"}).get_text() 
    valueDic['發行商'] = label.replace(u'\xa0', ' ').strip(' :\n')
    maker = bsSoup.find("div", {'id': "video_maker"}).find("td", {'class', "text"}).get_text() 
    valueDic['製作商'] = maker.replace(u'\xa0', ' ').strip(' :\n')
    length = bsSoup.find("div", {'id': "video_length"}).find("span", {'class', "text"}).get_text() 
    valueDic['長度'] = length
    date = bsSoup.find("div", {'id': "video_date"}).find("td", {'class', "text"}).get_text()
    valueDic['發行日期'] = date
    genre = bsSoup.find("div", {'id': "video_genres"}).find_all("span", {'class', "genre"})
    valueDic['類別'] = [x.get_text() for x in genre]
    cast = bsSoup.find("div", {'id': "video_cast"}).find_all("span", {'class', "star"})
    valueDic['演員'] = [x.get_text() for x in cast]

    valueDic['網站'] = 'JavLibrary'
    return valueDic


def getDicFromJavLibrary(avID):
    url = "http://www.javlibrary.com/tw/vl_searchbyid.php?keyword=" + avID
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavLib(soup)





def downloadImage(webPath, avID):
    ext = webPath[webPath.rfind('.'):]
    filename = localFolder + '/' + avID + ext
    r = requests.get(webPath, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)  

def dumpDic2Json(valueDic):
    ID = valueDic['序號']
    filename = localFolder + '/' + ID +'.json'
    f = open(filename, "w")
    json.dump(valueDic, f)
    f.close()
    
def loadDicFromJson(avID):
    filename = localFolder + '/' + avID +'.json'    
    f = open(filename, 'r')
    valueDic = json.load(f)
    f.close()
    return valueDic
    
def checkAvIdFileExist(avID):
    filename = localFolder + '/' + avID +'.json'
    if os.path.exists(filename):
        return True
    return False

def checkAvIdImgExist(avID):
    jpg = localFolder + '/' + avID +'.jpg'
    png = localFolder + '/' + avID +'.png'
    if os.path.exists(jpg) or os.path.exists(png):
        return True
    return False

def completeAvIDThruJavLibrary(avID, itemResult):
    vDic = {}
    #AvId status, 1 not download before, 2 not exist on Javhoo, 3 downloaded
    if not checkAvIdFileExist(avID):
        vDic = getDicFromJavLibrary(avID)
        if (len(vDic) == 0):
            itemResult[2] = 404
            return vDic; # not on every server
        if (len(vDic) == 2):
            print("999... multiple returns")
            itemResult[2] = 999
            return vDic
        itemResult[2] = 200
#        if vDic['序號'].lower() != avID.lower():
#            raise ValueError('序號 got and given not equal', vDic['序號'].lower(), avID.lower())
        dumpDic2Json(vDic)
    else:
        vDic = loadDicFromJson(avID)

    # check if img exist, otherwise download img
    if not checkAvIdImgExist(avID):
        downloadImage(vDic['封面圖'], vDic['序號'])
    
    return vDic

def completeAvID(avID, itemResult, showInfo = False):
    vDic = {}
    #AvId status, 1 not download before, 2 not exist on Javhoo, 3 downloaded
    if not checkAvIdFileExist(avID):
        vDic = getDicFromJavhoo(avID)
        if (len(vDic) == 0):
            itemResult[0] = 404
            vDic = getDicFromJavBus(avID)
            if (len(vDic) == 0):
                itemResult[1] = 404
                return vDic; # not on every server
            itemResult[1] = 200
        itemResult[0] = 200
        if vDic['序號'].lower() != avID.lower():
            raise ValueError('序號 got and given not equal', vDic['序號'].lower(), avID.lower())
        dumpDic2Json(vDic)
        if showInfo:
            print("Downloaded ", avID)
    else:
        vDic = loadDicFromJson(avID)
        if itemResult[0] == 0:
            itemResult[0] = 200
        elif itemResult[1] == 0:
            itemResult[1] = 200
        else:
            raise ValueError("incorrect itemResult", itemResult[0], itemResult[1])
            

    # check if img exist, otherwise download img
    if not checkAvIdImgExist(avID):
        downloadImage(vDic['封面圖'], vDic['序號'])
        if showInfo:
            print("Img Downloaded", vDic['序號'])
    
    return vDic


def dumpResult2Json(avIDResult):
    filename = 'avIDResult.json'
    f = open(filename, "w")
    json.dump(avIDResult, f)
    f.close()
    
def loadResultFromJson():
    filename = 'avIDResult.json'    
    f = open(filename, 'r')
    avIDResult = json.load(f)
    f.close()
    return avIDResult

def countAVResult(avIDResult):
    count0 = 0
    count404 = 0
    count200 = 0
    countWaitJavBus = 0
    countWaitJavLib = 0
    countManualJavLib = 0
    for key, value in avIDResult.items():
        if value[0] == 0 and value[1] == 0 and value[2] == 0:
            count0+=1
        elif value[0] == 200 or value[1] == 200 or value[2] == 200:
            count200+=1
        elif value[0] == 404 and value[1] == 404 and value[2] == 404:
            count404+=1
        elif value[0] == 404 and value[1] == 0:
            countWaitJavBus+=1
        elif value[0] == 404 and value[1] == 404 and value[2] == 0:
            countWaitJavLib+=1       
        elif value[2] == 999:
            countManualJavLib+=1          
        else:
            raise ValueError("Unknow value combination: ", value[0], value[1], value[2])
    print("AVIDResult: 0~", count0, " 200~", count200, " 404~", count404, 
          "wait trying JavBus~", countWaitJavBus, "wait trying JavLib~", countWaitJavLib, 
          "wait manual JavLib~", countManualJavLib)

from itertools import islice
from random import randint
import time
import random

    
def randomGet2(avIDResult, count):
    c_200 = 0
    c_404 = 0
    c_998 = 0
    c_999 = 0


    for i in range(count):
        key, value = random.choice(list(avIDResult.items()))

        t = 0
        print("{0: <10}".format(key), end='')
        if value[0] == 0 or (value[0] == 404 and value[1] == 0):    # not get from web before
            itemResult = avIDResult[key]
            vDic = completeAvID(key, itemResult)
            if len(vDic) > 0:
                if vDic['網站'] == 'Javhoo':
                    print("---- Jho downloaded, ", end='')
                elif vDic['網站'] == 'JavBus':
                    print("---- JBs downloaded, ", end='') 
                else:
                    raise ValueError("???? Where am I downloaded", vDic['網站'])
                c_200+=1
                t = random.uniform(0.5, 5.0)             
            else:
                c_404+=1
                print("@@@@ not on 2 Javs,  ", end='')
                t = random.uniform(0.5, 1.0)                   
        elif value[0] == 404 and value[1] == 404:
            print("#### 2 sites FAIL,   ", end='')
            c_998+=1
        else:
            print("!!!! already there,  ", end='')
            c_999+=1
        print("pause {0:.2f} seconds...".format(t))
        time.sleep(t)
    print("$$$$$ Batch for {0} items, {1} [200], {2} [404], {3} [exists], {4} [exists fail]."
          .format(count, c_200, c_404, c_999, c_998))


def serialGet(avIDResult, count):
    c_200 = 0
    c_404 = 0
    c_998 = 0
    c_999 = 0
    itemRemain = count

    for key, value in list(avIDResult.items()):
        t = 0
        if value[0] == 0 or (value[0] == 404 and value[1] == 0):    # not get from web before
            print("{0: <10}".format(key), end='')            
            itemResult = avIDResult[key]
            vDic = completeAvID(key, itemResult)
            if len(vDic) > 0:
                if vDic['網站'] == 'Javhoo':
                    print("---- Jho downloaded, ", end='')
                elif vDic['網站'] == 'JavBus':
                    print("---- JBs downloaded, ", end='') 
                else:
                    raise ValueError("???? Where am I downloaded", vDic['網站'])
                c_200+=1
                itemRemain-=1
                if (itemRemain == 0):
                    break
                t = random.uniform(0.5, 2.0)             
            else:
                c_404+=1
                print("@@@@ not on 2 Javs,  ", end='')
                t = random.uniform(0.1, 0.5)
            print("pause {0:.2f} seconds...".format(t))            
        elif value[0] == 404 and value[1] == 404:
#            print("{0: <10}#### 2 sites FAIL,   ".format(key), end='')
            c_998+=1
        else:
#            print("!!!! already there,  ", end='')
            c_999+=1
#        print("pause {0:.2f} seconds...".format(t))
        time.sleep(t)
    print("$$$$$ Batch for {0} items, {1} [200], {2} [404], {3} [exists], {4} [exists fail]."
          .format(count, c_200, c_404, c_999, c_998))

    
#another site :    https://www.arzon.jp/itemlist.html?t=&m=all&s=&q=JFD-07    這內容很好，但一開始有個同意頁，內容頁也較複雜
#also ano site:    https://javfree.me/?s=NTRD-037 
#another site :    http://www.javlib.com/cn/  這內容也很好，一開始也有頁，search好像不是從網址，要從網頁form
#another site :    https://onejav.com/

localFolder = '/Users/yiwenliu/Desktop/Projects/AV'

In [ ]:
avIDResult = loadResultFromJson()

In [ ]:
countAVResult(avIDResult)

In [ ]:
countAVResult(avIDResult)

In [ ]:
start_time = time.time()
for i in range(1):
    print("Round: {0}".format(i))
    serialGet(avIDResult, 30)
    dumpResult2Json(avIDResult)
countAVResult(avIDResult)
elapsed_time = time.time() - start_time
print("Time consumed: {0:.2f} secs".format(elapsed_time))

In [ ]:
countAVResult(avIDResult)

In [ ]:
# test JavLibrary
for key, value in avIDResult.items():
    if value[0] == 404 and value[1] == 404 and value[2] == 0:
        print(key, value, "try JavLibrary...")
        vDic = completeAvIDThruJavLibrary(key, value)
        print (vDic)
            
dumpResult2Json(avIDResult)
print("\nDone\n")

In [ ]:
dumpResult2Json(avIDResult)

In [ ]:
[y, t for t in y.lower() for y, x in avIDResult.items() if x[0] == 404 and x[1] == 404 and x[2] == 404]

In [ ]:
avIDResult["NDR-002"]

In [ ]:
vDic = completeAvIDThruJavLibrary('NDR-002', avIDResult['NDR-002'])
print (vDic)

In [ ]:
dumpResult2Json(avIDResult)

In [ ]:
completeAvID("PURE-004", avIDResult['PURE-004'])

In [ ]:
vDic = getDicFromJavBus("JMD-093")
print (vDic)

In [ ]:
avIDResult['NDR-002']

In [ ]:
checkAvIdFileExist('NDR-002')

In [ ]:
checkAvIdImgExist('NDR-002')

In [ ]:
import pickle
import json

def loadData():
    data = pickle.load(open("filelist.p", "rb"))
    return data

def loadIdFilename():
    data = json.load(open("avIDs_Filename.json", 'r'))
    return data

allfilesData = loadData()

idFilename = loadIdFilename()



In [ ]:
a

In [ ]:
total_files = []
for root, dir, files in os.walk(localFolder):
    for file in files:
        if not file.endswith("jpg") and not file.endswith("json"):
            print (file)
        if file.endswith("jpg"):
            jfile = file.replace('jpg', 'json')
            #print (root+'/'+jfile)
            if not os.path.exists(root + '/' + jfile):
                print(jfile, " not exists.")
                id = file[0:file.find('.')]
                print (id, end='')
                for item in idFilename:
                    if item[0] == id:
                        print (item[1])
                
                

In [ ]:
import difflib

total_files = []
jpgs = []
jsons = []

for root, dir, files in os.walk(localFolder):
    for file in files:
        if file.endswith("jpg"):
            jpgs.append(file[0:file.rfind('.')])
        if file.endswith("json"):
            jsons.append(file[0:file.rfind('.')])

for root, dir, files in os.walk(localFolder):
    for file in files:
        if file.endswith("jpg"):
            jfile = file.replace('jpg', 'json')
            if not os.path.exists(root + '/' + jfile):
                print('jpg:  ', file)
                s = difflib.get_close_matches(file[0:file.rfind('.')], jsons)
                print('json: ', s[0])
                os.rename(root+'/'+file, root+'/'+s[0]+'.jpg')
                
                